Table of contents:

01. Import Libraries

02. Define Foursquare Credentials

03. Define the city and get its latitude & longitude

04. Search for Hospitals & clean dataframe

05. Search for Hotels & clean dataframe

06. Search for Shopping Stores & clean dataframe

07. Search for Cafeteria & clean dataframe

08. Search for Restaurants & clean dataframe

09. Search for Cafeteria & clean dataframe

10. Search for Bars and Clubs  & clean dataframe  

11. Search for Tourist Centers & clean dataframe

12. Search for Spa & clean dataframe

13. Search for Parks & clean dataframe

14. Generate map 

15. Map for Basic facilities like Hotels, Hospital and Restaurant

16. Map to visualize neighbourhood including Cafeteria , Shopping Centers, Tourist Center, Spa and Park

### Importing Libraries

In [0]:
import numpy as np
import pandas as pd
import requests

from geopy.geocoders import Nominatim

from IPython.display import Image
from IPython.core.display import HTML

from pandas.io.json import json_normalize

import folium

In [0]:
# !conda install -c conda-forge geopy --yes

In [0]:
# !conda install -c conda-forge folium=0.5.0 --yes

###Define Foursquare Credentials

In [5]:
CLIENT_ID = 'YDDCQN3JYZL2WA41EACJN2XKXPMXJLA4ISAAUS4JTIINWTMA' # your Foursquare ID
CLIENT_SECRET = 'DQZVHL5SPJOZ1SVDMGQEMA5QM5X5XE1SUIENWGHIWXT1N0WB' # your Foursquare Secret
VERSION = '20180605'
LIMIT =30
print('Your credentails:')
print('Foursquare_ID: ' + CLIENT_ID)
print('Foursquare_Secret:' + CLIENT_SECRET)

Your credentails:
Foursquare_ID: YDDCQN3JYZL2WA41EACJN2XKXPMXJLA4ISAAUS4JTIINWTMA
Foursquare_Secret:DQZVHL5SPJOZ1SVDMGQEMA5QM5X5XE1SUIENWGHIWXT1N0WB


###Define the city and get its latitude & longitude

In [6]:
city = 'Tokyo'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

35.6828387 139.7594549


In [8]:
# search for hospitals
search_query = 'Hospital'
radius = 2000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=YDDCQN3JYZL2WA41EACJN2XKXPMXJLA4ISAAUS4JTIINWTMA&client_secret=DQZVHL5SPJOZ1SVDMGQEMA5QM5X5XE1SUIENWGHIWXT1N0WB&ll=35.6828387,139.7594549&v=20180605&query=Hospital&radius=2000&limit=30'

In [0]:
results = requests.get(url).json()

In [10]:
venues = results['response']['venues']

dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,4b7e3469f964a520ffe52fe3,Toranomon Hospital (虎の門病院),"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1577152417,False,虎ノ門2-2-2,35.668780,139.746678,"[{'label': 'display', 'lat': 35.66878028496117...",1945,105-0001,JP,港区,東京都,日本,"[虎ノ門2-2-2, 港区, 東京都, 105-0001, 日本]",NaN
1,540ac2af498e8d96ee17d47f,WATCH HOSPITAL 人形町店,"[{'id': '52f2ab2ebcbc57f1066b8b2e', 'name': 'W...",v-1577152417,False,日本橋人形町3-7-14,35.686308,139.782035,"[{'label': 'display', 'lat': 35.68630786572771...",2077,NaN,JP,東京,東京都,日本,"[日本橋人形町3-7-14, 中央区, 東京都, 日本]",NaN
2,4ba848b1f964a520e1d439e3,Hospital of the Imperial Household (宮内庁病院),"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1577152417,False,千代田1-2,35.686930,139.759784,"[{'label': 'display', 'lat': 35.68692982193012...",456,100-0001,JP,東京,東京都,日本,"[千代田1-2 (皇居東御苑), 千代田区, 東京都, 100-0001, 日本]",皇居東御苑
3,4b5a5103f964a520aabc28e3,Juntendo University Hospital (順天堂大学医学部附属 順天堂医院),"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1577152417,False,本郷3-1-3,35.702272,139.762504,"[{'label': 'display', 'lat': 35.70227176650229...",2180,113-8431,JP,東京,東京都,日本,"[本郷3-1-3, 文京区, 東京都, 113-8431, 日本]",NaN
4,4d15ea5c380a8eecb8ba85fa,National Cancer Center Hospital (国立がん研究センター中央病院),"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1577152417,False,築地5-1-1,35.665488,139.768159,"[{'label': 'display', 'lat': 35.66548753828965...",2085,104-0045,JP,東京,東京都,日本,"[築地5-1-1, 中央区, 東京都, 104-0045, 日本]",NaN


In [11]:
clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
clean_dataframe = dataframe.loc[:, clean_columns]

def get_category_type(row):
  try:
    categories_list = row['categories']
  except:
    categories_list = row['venue.categories']

  if len(categories_list) == 0:
    return None
  else:
    return categories_list[0]['name']

clean_dataframe['categories'] = clean_dataframe.apply(get_category_type, axis =1)

clean_dataframe.columns = [column.split('.')[-1] for column in clean_dataframe.columns]

clean_dataframe.head()



,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Toranomon Hospital (虎の門病院),Hospital,虎ノ門2-2-2,35.668780,139.746678,"[{'label': 'display', 'lat': 35.66878028496117...",1945,105-0001,JP,港区,東京都,日本,"[虎ノ門2-2-2, 港区, 東京都, 105-0001, 日本]",NaN,4b7e3469f964a520ffe52fe3
1,WATCH HOSPITAL 人形町店,Watch Shop,日本橋人形町3-7-14,35.686308,139.782035,"[{'label': 'display', 'lat': 35.68630786572771...",2077,NaN,JP,東京,東京都,日本,"[日本橋人形町3-7-14, 中央区, 東京都, 日本]",NaN,540ac2af498e8d96ee17d47f
2,Hospital of the Imperial Household (宮内庁病院),Hospital,千代田1-2,35.686930,139.759784,"[{'label': 'display', 'lat': 35.68692982193012...",456,100-0001,JP,東京,東京都,日本,"[千代田1-2 (皇居東御苑), 千代田区, 東京都, 100-0001, 日本]",皇居東御苑,4ba848b1f964a520e1d439e3
3,Juntendo University Hospital (順天堂大学医学部附属 順天堂医院),Hospital,本郷3-1-3,35.702272,139.762504,"[{'label': 'display', 'lat': 35.70227176650229...",2180,113-8431,JP,東京,東京都,日本,"[本郷3-1-3, 文京区, 東京都, 113-8431, 日本]",NaN,4b5a5103f964a520aabc28e3
4,National Cancer Center Hospital (国立がん研究センター中央病院),Hospital,築地5-1-1,35.665488,139.768159,"[{'label': 'display', 'lat': 35.66548753828965...",2085,104-0045,JP,東京,東京都,日本,"[築地5-1-1, 中央区, 東京都, 104-0045, 日本]",NaN,4d15ea5c380a8eecb8ba85fa


In [12]:
clean_dataframe2= clean_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'formattedAddress',\
                                        'labeledLatLngs', 'id', 'state', 'distance'], axis=1)
clean_dataframe2


,name,categories,address,lat,lng,postalCode
0,Toranomon Hospital (虎の門病院),Hospital,虎ノ門2-2-2,35.668780,139.746678,105-0001
1,WATCH HOSPITAL 人形町店,Watch Shop,日本橋人形町3-7-14,35.686308,139.782035,NaN
2,Hospital of the Imperial Household (宮内庁病院),Hospital,千代田1-2,35.686930,139.759784,100-0001
3,Juntendo University Hospital (順天堂大学医学部附属 順天堂医院),Hospital,本郷3-1-3,35.702272,139.762504,113-8431
4,National Cancer Center Hospital (国立がん研究センター中央病院),Hospital,築地5-1-1,35.665488,139.768159,104-0045
5,Jikei University School of Medicine Hospital (...,Hospital,西新橋3-19-18,35.663503,139.751265,105-8471
6,Nihon University Hospital (日本大学病院),Hospital,神田駿河台1-6,35.697421,139.762455,101-8309
7,Kyoundo Hospital (杏雲堂病院),Hospital,神田駿河台1-8,35.698375,139.762933,101-0062
8,Hanzomon Hospital (半蔵門病院),Hospital,麹町1-10-5,35.685427,139.742962,102-0083
9,Mitsui Memorial Hospital (三井記念病院),Hospital,神田和泉町1,35.700022,139.778912,101-8643


In [13]:
clean_dataframe3 = clean_dataframe2.dropna(axis=0, how = 'any', thresh=None, subset=None, inplace=False)
clean_dataframe3

,name,categories,address,lat,lng,postalCode
0,Toranomon Hospital (虎の門病院),Hospital,虎ノ門2-2-2,35.668780,139.746678,105-0001
2,Hospital of the Imperial Household (宮内庁病院),Hospital,千代田1-2,35.686930,139.759784,100-0001
3,Juntendo University Hospital (順天堂大学医学部附属 順天堂医院),Hospital,本郷3-1-3,35.702272,139.762504,113-8431
4,National Cancer Center Hospital (国立がん研究センター中央病院),Hospital,築地5-1-1,35.665488,139.768159,104-0045
5,Jikei University School of Medicine Hospital (...,Hospital,西新橋3-19-18,35.663503,139.751265,105-8471
6,Nihon University Hospital (日本大学病院),Hospital,神田駿河台1-6,35.697421,139.762455,101-8309
7,Kyoundo Hospital (杏雲堂病院),Hospital,神田駿河台1-8,35.698375,139.762933,101-0062
8,Hanzomon Hospital (半蔵門病院),Hospital,麹町1-10-5,35.685427,139.742962,102-0083
9,Mitsui Memorial Hospital (三井記念病院),Hospital,神田和泉町1,35.700022,139.778912,101-8643
10,Sanraku Hospital (三楽病院),Hospital,神田駿河台2-5,35.700489,139.761500,101-0062


In [17]:
df_hospital = clean_dataframe3.drop_duplicates(subset = 'name', keep = 'first')
df_hospital

,name,categories,address,lat,lng,postalCode
0,Toranomon Hospital (虎の門病院),Hospital,虎ノ門2-2-2,35.668780,139.746678,105-0001
2,Hospital of the Imperial Household (宮内庁病院),Hospital,千代田1-2,35.686930,139.759784,100-0001
3,Juntendo University Hospital (順天堂大学医学部附属 順天堂医院),Hospital,本郷3-1-3,35.702272,139.762504,113-8431
4,National Cancer Center Hospital (国立がん研究センター中央病院),Hospital,築地5-1-1,35.665488,139.768159,104-0045
5,Jikei University School of Medicine Hospital (...,Hospital,西新橋3-19-18,35.663503,139.751265,105-8471
6,Nihon University Hospital (日本大学病院),Hospital,神田駿河台1-6,35.697421,139.762455,101-8309
7,Kyoundo Hospital (杏雲堂病院),Hospital,神田駿河台1-8,35.698375,139.762933,101-0062
8,Hanzomon Hospital (半蔵門病院),Hospital,麹町1-10-5,35.685427,139.742962,102-0083
9,Mitsui Memorial Hospital (三井記念病院),Hospital,神田和泉町1,35.700022,139.778912,101-8643
10,Sanraku Hospital (三楽病院),Hospital,神田駿河台2-5,35.700489,139.761500,101-0062


##For Hotels

In [18]:
search_query = 'Hotel'
radius = 2000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=YDDCQN3JYZL2WA41EACJN2XKXPMXJLA4ISAAUS4JTIINWTMA&client_secret=DQZVHL5SPJOZ1SVDMGQEMA5QM5X5XE1SUIENWGHIWXT1N0WB&ll=35.6828387,139.7594549&v=20180605&query=Hotel&radius=2000&limit=30'

In [0]:
# Send the GET Request and examine the results
results = requests.get(url).json()
#results

In [20]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood
0,4b05879af964a520ac9b22e3,Palace Hotel Tokyo (パレスホテル東京),"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1577152488,False,丸の内1-1-1,35.684644,139.761302,"[{'label': 'display', 'lat': 35.684644, 'lng':...",261,100-0005,JP,東京,東京都,日本,"[丸の内1-1-1, 千代田区, 東京都, 100-0005, 日本]",NaN,NaN
1,4b0587a6f964a5203e9e22e3,Tokyo Station (東京駅),"[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",v-1577152488,False,丸の内1,35.681190,139.767230,"[{'label': 'display', 'lat': 35.68119, 'lng': ...",726,100-0005,JP,東京,東京都,日本,"[丸の内1, 千代田区, 東京都, 100-0005, 日本]",NaN,NaN
2,4b058799f964a5208d9b22e3,Imperial Hotel Tokyo (帝国ホテル 東京),"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1577152488,False,内幸町1-1-1,35.672468,139.758322,"[{'label': 'display', 'lat': 35.67246820552319...",1158,100-8558,JP,東京,東京都,日本,"[内幸町1-1-1, 千代田区, 東京都, 100-8558, 日本]",NaN,NaN
3,50b58b02e4b0be8d47ad6128,Hotel Arcade (ホテルアーケード),"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",v-1577152488,False,丸の内1-1-1,35.684540,139.761320,"[{'label': 'display', 'lat': 35.684539726631, ...",253,NaN,JP,東京,東京都,日本,"[丸の内1-1-1 (パレスホテル B1F), 千代田区, 東京都, 日本]",パレスホテル B1F,NaN
4,4bb00caff964a52003343ce3,Tokyo Dome Hotel (東京ドームホテル),"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1577152488,False,後楽1-3-61,35.703617,139.753470,"[{'label': 'display', 'lat': 35.70361674159982...",2375,112-8562,JP,東京,東京都,日本,"[後楽1-3-61, 文京区, 東京都, 112-8562, 日本]",NaN,NaN


###Cleaning Dataframe

In [21]:
# keep only columns that include venue name, and anything that is associated with location
clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_dataframe = dataframe.loc[:,clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
clean_dataframe['categories'] = clean_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_dataframe.columns = [column.split('.')[-1] for column in clean_dataframe.columns]

clean_dataframe.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Palace Hotel Tokyo (パレスホテル東京),Hotel,丸の内1-1-1,35.684644,139.761302,"[{'label': 'display', 'lat': 35.684644, 'lng':...",261,100-0005,JP,東京,東京都,日本,"[丸の内1-1-1, 千代田区, 東京都, 100-0005, 日本]",NaN,NaN,4b05879af964a520ac9b22e3
1,Tokyo Station (東京駅),Train Station,丸の内1,35.681190,139.767230,"[{'label': 'display', 'lat': 35.68119, 'lng': ...",726,100-0005,JP,東京,東京都,日本,"[丸の内1, 千代田区, 東京都, 100-0005, 日本]",NaN,NaN,4b0587a6f964a5203e9e22e3
2,Imperial Hotel Tokyo (帝国ホテル 東京),Hotel,内幸町1-1-1,35.672468,139.758322,"[{'label': 'display', 'lat': 35.67246820552319...",1158,100-8558,JP,東京,東京都,日本,"[内幸町1-1-1, 千代田区, 東京都, 100-8558, 日本]",NaN,NaN,4b058799f964a5208d9b22e3
3,Hotel Arcade (ホテルアーケード),Shopping Mall,丸の内1-1-1,35.684540,139.761320,"[{'label': 'display', 'lat': 35.684539726631, ...",253,NaN,JP,東京,東京都,日本,"[丸の内1-1-1 (パレスホテル B1F), 千代田区, 東京都, 日本]",パレスホテル B1F,NaN,50b58b02e4b0be8d47ad6128
4,Tokyo Dome Hotel (東京ドームホテル),Hotel,後楽1-3-61,35.703617,139.753470,"[{'label': 'display', 'lat': 35.70361674159982...",2375,112-8562,JP,東京,東京都,日本,"[後楽1-3-61, 文京区, 東京都, 112-8562, 日本]",NaN,NaN,4bb00caff964a52003343ce3


In [22]:
# delete unnecessary columns
clean_dataframe2= clean_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs','neighborhood', 'id'], axis=1)
clean_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Palace Hotel Tokyo (パレスホテル東京),Hotel,丸の内1-1-1,35.684644,139.761302,100-0005,東京都
1,Tokyo Station (東京駅),Train Station,丸の内1,35.681190,139.767230,100-0005,東京都
2,Imperial Hotel Tokyo (帝国ホテル 東京),Hotel,内幸町1-1-1,35.672468,139.758322,100-8558,東京都
3,Hotel Arcade (ホテルアーケード),Shopping Mall,丸の内1-1-1,35.684540,139.761320,NaN,東京都
4,Tokyo Dome Hotel (東京ドームホテル),Hotel,後楽1-3-61,35.703617,139.753470,112-8562,東京都
5,Four Seasons Hotel Tokyo at Marunouchi (フォーシーズ...,Hotel,丸の内1-11-1,35.677912,139.766808,100-6277,東京都
6,Hotel Ryumeikan Tokyo (ホテル龍名館東京),Hotel,八重洲1-3-22,35.682375,139.770832,101-0028,東京都
7,Hotel the Celestine Ginza (ホテル ザ セレスティン銀座),Hotel,銀座8-4-22,35.669172,139.759801,104-0061,東京都
8,Hotel Main Kanda (ホテルメイン 神田),Hotel,内神田3-1-6,35.689416,139.769730,101-0047,東京都
9,Hotel Villa Fontaine Otemachi (ヴィラフォンテーヌ 大手町),Hotel,内神田1-8-8,35.690651,139.767015,101-0047,東京都


In [23]:
# delete rows with none values
clean_dataframe3 = clean_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_dataframe3

,name,categories,address,lat,lng,postalCode,state
0,Palace Hotel Tokyo (パレスホテル東京),Hotel,丸の内1-1-1,35.684644,139.761302,100-0005,東京都
1,Tokyo Station (東京駅),Train Station,丸の内1,35.681190,139.767230,100-0005,東京都
2,Imperial Hotel Tokyo (帝国ホテル 東京),Hotel,内幸町1-1-1,35.672468,139.758322,100-8558,東京都
4,Tokyo Dome Hotel (東京ドームホテル),Hotel,後楽1-3-61,35.703617,139.753470,112-8562,東京都
5,Four Seasons Hotel Tokyo at Marunouchi (フォーシーズ...,Hotel,丸の内1-11-1,35.677912,139.766808,100-6277,東京都
6,Hotel Ryumeikan Tokyo (ホテル龍名館東京),Hotel,八重洲1-3-22,35.682375,139.770832,101-0028,東京都
7,Hotel the Celestine Ginza (ホテル ザ セレスティン銀座),Hotel,銀座8-4-22,35.669172,139.759801,104-0061,東京都
8,Hotel Main Kanda (ホテルメイン 神田),Hotel,内神田3-1-6,35.689416,139.769730,101-0047,東京都
9,Hotel Villa Fontaine Otemachi (ヴィラフォンテーヌ 大手町),Hotel,内神田1-8-8,35.690651,139.767015,101-0047,東京都
10,Hotel Mystays Premier Akasaka (ホテルマイステイズプレミア赤坂),Hotel,赤坂2-17-54,35.670134,139.738332,107-0052,東京都


In [24]:
# delete rows which its category is not Hotel or Event Space
array= ['Hotel', 'Event Space']
hotel_dataframe= clean_dataframe3.loc[clean_dataframe3['categories'].isin(array)]
hotel_dataframe

,name,categories,address,lat,lng,postalCode,state
0,Palace Hotel Tokyo (パレスホテル東京),Hotel,丸の内1-1-1,35.684644,139.761302,100-0005,東京都
2,Imperial Hotel Tokyo (帝国ホテル 東京),Hotel,内幸町1-1-1,35.672468,139.758322,100-8558,東京都
4,Tokyo Dome Hotel (東京ドームホテル),Hotel,後楽1-3-61,35.703617,139.753470,112-8562,東京都
5,Four Seasons Hotel Tokyo at Marunouchi (フォーシーズ...,Hotel,丸の内1-11-1,35.677912,139.766808,100-6277,東京都
6,Hotel Ryumeikan Tokyo (ホテル龍名館東京),Hotel,八重洲1-3-22,35.682375,139.770832,101-0028,東京都
7,Hotel the Celestine Ginza (ホテル ザ セレスティン銀座),Hotel,銀座8-4-22,35.669172,139.759801,104-0061,東京都
8,Hotel Main Kanda (ホテルメイン 神田),Hotel,内神田3-1-6,35.689416,139.769730,101-0047,東京都
9,Hotel Villa Fontaine Otemachi (ヴィラフォンテーヌ 大手町),Hotel,内神田1-8-8,35.690651,139.767015,101-0047,東京都
10,Hotel Mystays Premier Akasaka (ホテルマイステイズプレミア赤坂),Hotel,赤坂2-17-54,35.670134,139.738332,107-0052,東京都
14,Solaria Nishitetsu Hotel Ginza (ソラリア西鉄ホテル銀座),Hotel,銀座4-9-2,35.670863,139.767403,104-0061,東京都


In [25]:
df_hotels = hotel_dataframe.drop_duplicates(subset='name', keep="first")
df_hotels

,name,categories,address,lat,lng,postalCode,state
0,Palace Hotel Tokyo (パレスホテル東京),Hotel,丸の内1-1-1,35.684644,139.761302,100-0005,東京都
2,Imperial Hotel Tokyo (帝国ホテル 東京),Hotel,内幸町1-1-1,35.672468,139.758322,100-8558,東京都
4,Tokyo Dome Hotel (東京ドームホテル),Hotel,後楽1-3-61,35.703617,139.753470,112-8562,東京都
5,Four Seasons Hotel Tokyo at Marunouchi (フォーシーズ...,Hotel,丸の内1-11-1,35.677912,139.766808,100-6277,東京都
6,Hotel Ryumeikan Tokyo (ホテル龍名館東京),Hotel,八重洲1-3-22,35.682375,139.770832,101-0028,東京都
7,Hotel the Celestine Ginza (ホテル ザ セレスティン銀座),Hotel,銀座8-4-22,35.669172,139.759801,104-0061,東京都
8,Hotel Main Kanda (ホテルメイン 神田),Hotel,内神田3-1-6,35.689416,139.769730,101-0047,東京都
9,Hotel Villa Fontaine Otemachi (ヴィラフォンテーヌ 大手町),Hotel,内神田1-8-8,35.690651,139.767015,101-0047,東京都
10,Hotel Mystays Premier Akasaka (ホテルマイステイズプレミア赤坂),Hotel,赤坂2-17-54,35.670134,139.738332,107-0052,東京都
14,Solaria Nishitetsu Hotel Ginza (ソラリア西鉄ホテル銀座),Hotel,銀座4-9-2,35.670863,139.767403,104-0061,東京都


## Places for Shopping

In [26]:
# search for Shopping
search_query = 'Shopping'
radius = 5000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=YDDCQN3JYZL2WA41EACJN2XKXPMXJLA4ISAAUS4JTIINWTMA&client_secret=DQZVHL5SPJOZ1SVDMGQEMA5QM5X5XE1SUIENWGHIWXT1N0WB&ll=35.6828387,139.7594549&v=20180605&query=Shopping&radius=5000&limit=30'

In [0]:
# Send the GET Request and examine the results
shopping_results = requests.get(url).json()
# shopping_results

In [28]:
venues = shopping_results['response']['venues']

# tranform venues into a dataframe
Shopping_dataframe = json_normalize(venues)
Shopping_dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,location.crossStreet
0,4b75371cf964a52068012ee3,Yaesu Shopping Mall (八重洲地下街),"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",v-1577152509,False,八重洲2-1,35.680048,139.768465,"[{'label': 'display', 'lat': 35.6800478644524,...",871,104-0028,JP,東京,東京都,日本,"[八重洲2-1, 中央区, 東京都, 104-0028, 日本]",NaN,NaN
1,4b35706ff964a5202b2d25e3,Seibu (西武渋谷店),"[{'id': '4bf58dd8d48988d1f6941735', 'name': 'D...",v-1577152509,False,宇田川町21-1,35.660328,139.700308,"[{'label': 'display', 'lat': 35.66032765766402...",5906,150-8330,JP,東京,東京都,日本,"[宇田川町21-1, 渋谷区, 東京都, 150-8330, 日本]",宇田川町,NaN
2,4b5a8df8f964a52023cb28e3,Ameya Yokocho (アメヤ横丁),"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",v-1577152509,False,上野4,35.709964,139.774581,"[{'label': 'display', 'lat': 35.7099644999719,...",3314,110-0005,JP,東京,東京都,日本,"[上野4, 台東区, 東京都, 110-0005, 日本]",上野,NaN
3,4eb4ecfa823131626f07a6d7,銀座中央通り,"[{'id': '4bf58dd8d48988d1f9931735', 'name': 'R...",v-1577152509,False,銀座1~8,35.669918,139.763657,"[{'label': 'display', 'lat': 35.66991800926114...",1487,104-0061,JP,東京,東京都,日本,"[銀座1~8, 中央区, 東京都, 104-0061, 日本]",NaN,NaN
4,4c9da26a54c8a1cde7f5854b,Smoking Area - Yaesu Shopping Mall (八重洲地下街喫煙所),"[{'id': '4bf58dd8d48988d164941735', 'name': 'P...",v-1577152509,False,八重洲2-1,35.679404,139.771424,"[{'label': 'display', 'lat': 35.679403523213, ...",1147,104-0028,JP,中央区,東京都,日本,"[八重洲2-1 (八重洲地下街), 中央区, 東京都, 104-0028, 日本]",NaN,八重洲地下街


### Cleaning Dataframes

In [29]:
# keep only columns that include venue name, and anything that is associated with location
Shopping_clean_columns = ['name', 'categories'] + [col for col in Shopping_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Shopping_dataframe = Shopping_dataframe.loc[:,Shopping_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list5 = row['categories']
    except:
        categories_list5 = row['venue.categories']
        
    if len(categories_list5) == 0:
        return None
    else:
        return categories_list5[0]['name']

# filter the category for each row
clean_Shopping_dataframe['categories'] = clean_Shopping_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Shopping_dataframe.columns = [column.split('.')[-1] for column in clean_Shopping_dataframe.columns]

clean_Shopping_dataframe.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,crossStreet,id
0,Yaesu Shopping Mall (八重洲地下街),Shopping Mall,八重洲2-1,35.680048,139.768465,"[{'label': 'display', 'lat': 35.6800478644524,...",871,104-0028,JP,東京,東京都,日本,"[八重洲2-1, 中央区, 東京都, 104-0028, 日本]",NaN,NaN,4b75371cf964a52068012ee3
1,Seibu (西武渋谷店),Department Store,宇田川町21-1,35.660328,139.700308,"[{'label': 'display', 'lat': 35.66032765766402...",5906,150-8330,JP,東京,東京都,日本,"[宇田川町21-1, 渋谷区, 東京都, 150-8330, 日本]",宇田川町,NaN,4b35706ff964a5202b2d25e3
2,Ameya Yokocho (アメヤ横丁),Shopping Mall,上野4,35.709964,139.774581,"[{'label': 'display', 'lat': 35.7099644999719,...",3314,110-0005,JP,東京,東京都,日本,"[上野4, 台東区, 東京都, 110-0005, 日本]",上野,NaN,4b5a8df8f964a52023cb28e3
3,銀座中央通り,Road,銀座1~8,35.669918,139.763657,"[{'label': 'display', 'lat': 35.66991800926114...",1487,104-0061,JP,東京,東京都,日本,"[銀座1~8, 中央区, 東京都, 104-0061, 日本]",NaN,NaN,4eb4ecfa823131626f07a6d7
4,Smoking Area - Yaesu Shopping Mall (八重洲地下街喫煙所),Plaza,八重洲2-1,35.679404,139.771424,"[{'label': 'display', 'lat': 35.679403523213, ...",1147,104-0028,JP,中央区,東京都,日本,"[八重洲2-1 (八重洲地下街), 中央区, 東京都, 104-0028, 日本]",NaN,八重洲地下街,4c9da26a54c8a1cde7f5854b


In [30]:
# delete unnecessary columns
clean_Shopping_dataframe2= clean_Shopping_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_Shopping_dataframe2

,name,categories,address,lat,lng,postalCode,state,neighborhood
0,Yaesu Shopping Mall (八重洲地下街),Shopping Mall,八重洲2-1,35.680048,139.768465,104-0028,東京都,NaN
1,Seibu (西武渋谷店),Department Store,宇田川町21-1,35.660328,139.700308,150-8330,東京都,宇田川町
2,Ameya Yokocho (アメヤ横丁),Shopping Mall,上野4,35.709964,139.774581,110-0005,東京都,上野
3,銀座中央通り,Road,銀座1~8,35.669918,139.763657,104-0061,東京都,NaN
4,Smoking Area - Yaesu Shopping Mall (八重洲地下街喫煙所),Plaza,八重洲2-1,35.679404,139.771424,104-0028,東京都,NaN
5,Starbucks,Coffee Shop,麻布十番1-11-14,35.655715,139.736027,106-0045,東京都,NaN
6,月島西仲通り商店街,Shopping Mall,月島,35.663427,139.781307,104-0052,東京都,月島
7,Nakamise Shopping Street (仲見世商店街),Shopping Mall,浅草1,35.711957,139.796458,111-0032,東京都,NaN
8,Yanaka Ginza Shotengai (谷中銀座商店街),Shopping Mall,谷中3,35.727645,139.767033,110-0001,東京都,谷中
9,Azabujuban Shotengai (麻布十番商店街),Shopping Mall,麻布十番,35.655447,139.736854,106-0045,東京都,NaN


In [31]:
# delete rows which its category is not Shopping Mall
df_Shopping = clean_Shopping_dataframe2[clean_Shopping_dataframe2.categories == 'Shopping Mall']
df_Shopping

,name,categories,address,lat,lng,postalCode,state,neighborhood
0,Yaesu Shopping Mall (八重洲地下街),Shopping Mall,八重洲2-1,35.680048,139.768465,104-0028,東京都,NaN
2,Ameya Yokocho (アメヤ横丁),Shopping Mall,上野4,35.709964,139.774581,110-0005,東京都,上野
6,月島西仲通り商店街,Shopping Mall,月島,35.663427,139.781307,104-0052,東京都,月島
7,Nakamise Shopping Street (仲見世商店街),Shopping Mall,浅草1,35.711957,139.796458,111-0032,東京都,NaN
8,Yanaka Ginza Shotengai (谷中銀座商店街),Shopping Mall,谷中3,35.727645,139.767033,110-0001,東京都,谷中
9,Azabujuban Shotengai (麻布十番商店街),Shopping Mall,麻布十番,35.655447,139.736854,106-0045,東京都,NaN
11,Asakusa Underground Shopping Street (浅草地下街),Shopping Mall,浅草1-1-12,35.710890,139.797807,111-0033,東京都,NaN
12,Shin-Nakamise Shopping Street (新仲見世商店街),Shopping Mall,浅草,35.711888,139.795528,111-0032,東京都,NaN
14,穏田商店会,Shopping Mall,神宮前6丁目,35.666601,139.705784,NaN,東京都,NaN


## Looking for Cafeteria

In [34]:
#search fro cafeteria
search_query = 'Cafeteria'
radius = 5000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=YDDCQN3JYZL2WA41EACJN2XKXPMXJLA4ISAAUS4JTIINWTMA&client_secret=DQZVHL5SPJOZ1SVDMGQEMA5QM5X5XE1SUIENWGHIWXT1N0WB&ll=35.6828387,139.7594549&v=20180605&query=Cafeteria&radius=5000&limit=30'

In [0]:
# Send the GET Request and examine the results
cafeteria_results = requests.get(url).json()
#cafeteria_results

In [36]:
#assign relevant part of JSON to venues
venues = cafeteria_results['response']['venues']

# tranform venues into a dataframe
Cafeteria_dataframe = json_normalize(venues)
Cafeteria_dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood
0,4e380bb222713b80ba021ccb,Shibaura Institute of Technology Cafeteria (芝浦...,"[{'id': '4bf58dd8d48988d1a1941735', 'name': 'C...",v-1577152549,False,豊洲3-7-5,芝浦工業大学 交流棟 3F,35.660395,139.794331,"[{'label': 'display', 'lat': 35.66039521472596...",4023,135-8548,JP,東京,東京都,日本,"[豊洲3-7-5 (芝浦工業大学 交流棟 3F), 江東区, 東京都, 135-8548, 日本]",NaN
1,4e4e0083b61cf637a4e182cc,KDDI CAFETERIA MENDON屋,"[{'id': '4bf58dd8d48988d128941735', 'name': 'C...",v-1577152549,False,西新宿2-3-2,KDDIビル B1F,35.687701,139.695336,"[{'label': 'display', 'lat': 35.68770149692163...",5822,163-8003,JP,東京,東京都,日本,"[西新宿2-3-2 (KDDIビル B1F), 新宿区, 東京都, 163-8003, 日本]",NaN
2,4b6a434bf964a520c9cf2be3,Cafétéria Carré,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1577152549,False,六本木7-22-2,国立新美術館 B1F,35.664747,139.726748,"[{'label': 'display', 'lat': 35.66474683248830...",3578,106-0032,JP,東京,東京都,日本,"[六本木7-22-2 (国立新美術館 B1F), 港区, 東京都, 106-0032, 日本]",NaN
3,51a58434498edd2edc51767d,DigitalHollywood Cafeteria,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1577152549,False,NaN,NaN,35.698795,139.766273,"[{'label': 'display', 'lat': 35.698795, 'lng':...",1880,NaN,JP,NaN,NaN,日本,[日本],NaN
4,58be359f3ef0f62921fb1ba5,Cafeteria Spice Jaya,"[{'id': '4bf58dd8d48988d142941735', 'name': 'A...",v-1577152549,False,松が谷4-26-14,NaN,35.718183,139.788569,"[{'label': 'display', 'lat': 35.718183, 'lng':...",4733,NaN,JP,台東区,東京都,日本,"[松が谷4-26-14, 台東区, 東京都, 日本]",NaN


### Cafeteria Dataframe Cleaning

In [37]:
# keep only columns that include venue name, and anything that is associated with location
Cafeteria_clean_columns = ['name', 'categories'] + [col for col in Cafeteria_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Cafeteria_dataframe = Cafeteria_dataframe.loc[:,Cafeteria_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list4 = row['categories']
    except:
        categories_list4 = row['venue.categories']
        
    if len(categories_list4) == 0:
        return None
    else:
        return categories_list4[0]['name']

# filter the category for each row
clean_Cafeteria_dataframe['categories'] = clean_Cafeteria_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Cafeteria_dataframe.columns = [column.split('.')[-1] for column in clean_Cafeteria_dataframe.columns]

clean_Cafeteria_dataframe.head()


,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Shibaura Institute of Technology Cafeteria (芝浦...,College Cafeteria,豊洲3-7-5,芝浦工業大学 交流棟 3F,35.660395,139.794331,"[{'label': 'display', 'lat': 35.66039521472596...",4023,135-8548,JP,東京,東京都,日本,"[豊洲3-7-5 (芝浦工業大学 交流棟 3F), 江東区, 東京都, 135-8548, 日本]",NaN,4e380bb222713b80ba021ccb
1,KDDI CAFETERIA MENDON屋,Cafeteria,西新宿2-3-2,KDDIビル B1F,35.687701,139.695336,"[{'label': 'display', 'lat': 35.68770149692163...",5822,163-8003,JP,東京,東京都,日本,"[西新宿2-3-2 (KDDIビル B1F), 新宿区, 東京都, 163-8003, 日本]",NaN,4e4e0083b61cf637a4e182cc
2,Cafétéria Carré,Café,六本木7-22-2,国立新美術館 B1F,35.664747,139.726748,"[{'label': 'display', 'lat': 35.66474683248830...",3578,106-0032,JP,東京,東京都,日本,"[六本木7-22-2 (国立新美術館 B1F), 港区, 東京都, 106-0032, 日本]",NaN,4b6a434bf964a520c9cf2be3
3,DigitalHollywood Cafeteria,Café,NaN,NaN,35.698795,139.766273,"[{'label': 'display', 'lat': 35.698795, 'lng':...",1880,NaN,JP,NaN,NaN,日本,[日本],NaN,51a58434498edd2edc51767d
4,Cafeteria Spice Jaya,Asian Restaurant,松が谷4-26-14,NaN,35.718183,139.788569,"[{'label': 'display', 'lat': 35.718183, 'lng':...",4733,NaN,JP,台東区,東京都,日本,"[松が谷4-26-14, 台東区, 東京都, 日本]",NaN,58be359f3ef0f62921fb1ba5


In [38]:
# delete unnecessary columns
clean_Cafeteria_dataframe2= clean_Cafeteria_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)
clean_Cafeteria_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Shibaura Institute of Technology Cafeteria (芝浦...,College Cafeteria,豊洲3-7-5,35.660395,139.794331,135-8548,東京都
1,KDDI CAFETERIA MENDON屋,Cafeteria,西新宿2-3-2,35.687701,139.695336,163-8003,東京都
2,Cafétéria Carré,Café,六本木7-22-2,35.664747,139.726748,106-0032,東京都
3,DigitalHollywood Cafeteria,Café,NaN,35.698795,139.766273,NaN,NaN
4,Cafeteria Spice Jaya,Asian Restaurant,松が谷4-26-14,35.718183,139.788569,NaN,東京都
5,明治大学 スカイラウンジ暁,College Cafeteria,神田駿河台1-1,35.697456,139.761451,101-8301,東京都
6,Esprit kitchen Cafeteria,Café,六本木3-9-8,35.663479,139.733603,106-0032,東京都
7,Cafeteria (食堂),College Cafeteria,高輪2-2-1-32,35.638425,139.736883,108-0074,東京都
8,慶應義塾大学 The Cafeteria,Restaurant,三田2-15-45,35.648560,139.742990,NaN,NaN
9,KDDI CAFETERIA Deli Kitchen,Diner,西新宿2-3-3,35.687979,139.694979,NaN,東京都


In [39]:
# delete rows with none values
df_Cafeteria = clean_Cafeteria_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Cafeteria

,name,categories,address,lat,lng,postalCode,state
0,Shibaura Institute of Technology Cafeteria (芝浦...,College Cafeteria,豊洲3-7-5,35.660395,139.794331,135-8548,東京都
1,KDDI CAFETERIA MENDON屋,Cafeteria,西新宿2-3-2,35.687701,139.695336,163-8003,東京都
2,Cafétéria Carré,Café,六本木7-22-2,35.664747,139.726748,106-0032,東京都
5,明治大学 スカイラウンジ暁,College Cafeteria,神田駿河台1-1,35.697456,139.761451,101-8301,東京都
6,Esprit kitchen Cafeteria,Café,六本木3-9-8,35.663479,139.733603,106-0032,東京都
7,Cafeteria (食堂),College Cafeteria,高輪2-2-1-32,35.638425,139.736883,108-0074,東京都
12,カフェテリア11,Cafeteria,後楽1-3-61,35.706584,139.753072,112-8575,東京都
14,都庁第一本庁舎職員食堂,Corporate Cafeteria,西新宿2-8-1,35.689480,139.691807,163-8001,東京都
15,シーガル,College Cafeteria,東新橋1-8-1,35.664244,139.761763,105-7001,東京都
16,West Garden Cafeteria (西園食堂),Cafeteria,上野公園9-83,35.714574,139.768874,110-8711,東京都


## Looking for Bars & Clubs

In [41]:
# search for Restaurants
search_query = 'Bars and Clubs'
radius = 5000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=YDDCQN3JYZL2WA41EACJN2XKXPMXJLA4ISAAUS4JTIINWTMA&client_secret=DQZVHL5SPJOZ1SVDMGQEMA5QM5X5XE1SUIENWGHIWXT1N0WB&ll=35.6828387,139.7594549&v=20180605&query=Bars and Clubs&radius=5000&limit=30'

In [0]:
# Send the GET Request and examine the results
bar_results = requests.get(url).json()


In [43]:
# assign relevant part of JSON to venues
venues = bar_results['response']['venues']

# tranform venues into a dataframe
bars_dataframe = json_normalize(venues)
bars_dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood
0,4f3bbf53be772a0209216c9d,Darts&Bar share,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",v-1577152584,False,上野2-1-9,35.708335,139.772689,"[{'label': 'display', 'lat': 35.70833544628686...",3080,110-0005,JP,台東区,東京都,日本,"[上野2-1-9, 台東区, 東京都, 110-0005, 日本]",NaN,NaN
1,52e27cff11d2c04afba3b970,Cafe&SportsBar sol,"[{'id': '4bf58dd8d48988d11d941735', 'name': 'S...",v-1577152584,False,神田佐久間町4-9 みちのくビルB1,35.692899,139.771011,"[{'label': 'display', 'lat': 35.69289862058233...",1531,NaN,JP,千代田区,東京都,日本,"[神田佐久間町4-9 みちのくビルB1, 千代田区, 東京都, 日本]",NaN,NaN
2,4c64b93e94160f4718be2d17,Italian Restrant & Bar Spoon,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1577152584,False,日本橋茅場町2-4-3,35.678733,139.778021,"[{'label': 'display', 'lat': 35.67873266385969...",1739,NaN,JP,中央区,東京都,日本,"[日本橋茅場町2-4-3, 中央区, 東京都, 日本]",NaN,NaN
3,4e13005152b1655efa534dac,Jazz & Cafe Bar Small Hours,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",v-1577152584,False,神田神保町1-62-4,35.696879,139.757973,"[{'label': 'display', 'lat': 35.69687892670882...",1568,101-0051,JP,東京,東京都,日本,"[神田神保町1-62-4 (和光ビルジング1F), 千代田区, 東京都, 101-0051,...",和光ビルジング1F,NaN
4,4e152a59e4cded561126526f,CURRY&SOUTH ASIAN BAR SHAGORIKA,"[{'id': '4bf58dd8d48988d142941735', 'name': 'A...",v-1577152584,False,NaN,35.695417,139.763466,"[{'label': 'display', 'lat': 35.695417, 'lng':...",1446,NaN,JP,東京,東京都,日本,"[千代田区, 東京都, 日本]",NaN,NaN


### Cleaning Dataframe

In [44]:
# keep only columns that include venue name, and anything that is associated with location
bars_clean_columns = ['name', 'categories'] + [col for col in bars_dataframe.columns if col.startswith('location.')]+ ['id']
clean_bars_dataframe = bars_dataframe.loc[:, bars_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list3 = row['categories']
    except:
        categories_list3 = row['venue.categories']
        
    if len(categories_list3) == 0:
        return None
    else:
        return categories_list3[0]['name']

# filter the category for each row
clean_bars_dataframe['categories'] = clean_bars_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_bars_dataframe.columns = [column.split('.')[-1] for column in clean_bars_dataframe.columns]

clean_bars_dataframe.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Darts&Bar share,Bar,上野2-1-9,35.708335,139.772689,"[{'label': 'display', 'lat': 35.70833544628686...",3080,110-0005,JP,台東区,東京都,日本,"[上野2-1-9, 台東区, 東京都, 110-0005, 日本]",NaN,NaN,4f3bbf53be772a0209216c9d
1,Cafe&SportsBar sol,Sports Bar,神田佐久間町4-9 みちのくビルB1,35.692899,139.771011,"[{'label': 'display', 'lat': 35.69289862058233...",1531,NaN,JP,千代田区,東京都,日本,"[神田佐久間町4-9 みちのくビルB1, 千代田区, 東京都, 日本]",NaN,NaN,52e27cff11d2c04afba3b970
2,Italian Restrant & Bar Spoon,Italian Restaurant,日本橋茅場町2-4-3,35.678733,139.778021,"[{'label': 'display', 'lat': 35.67873266385969...",1739,NaN,JP,中央区,東京都,日本,"[日本橋茅場町2-4-3, 中央区, 東京都, 日本]",NaN,NaN,4c64b93e94160f4718be2d17
3,Jazz & Cafe Bar Small Hours,Bar,神田神保町1-62-4,35.696879,139.757973,"[{'label': 'display', 'lat': 35.69687892670882...",1568,101-0051,JP,東京,東京都,日本,"[神田神保町1-62-4 (和光ビルジング1F), 千代田区, 東京都, 101-0051,...",和光ビルジング1F,NaN,4e13005152b1655efa534dac
4,CURRY&SOUTH ASIAN BAR SHAGORIKA,Asian Restaurant,NaN,35.695417,139.763466,"[{'label': 'display', 'lat': 35.695417, 'lng':...",1446,NaN,JP,東京,東京都,日本,"[千代田区, 東京都, 日本]",NaN,NaN,4e152a59e4cded561126526f


In [45]:
# delete unnecessary columns
clean_bars_dataframe2= clean_bars_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)
clean_bars_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Darts&Bar share,Bar,上野2-1-9,35.708335,139.772689,110-0005,東京都
1,Cafe&SportsBar sol,Sports Bar,神田佐久間町4-9 みちのくビルB1,35.692899,139.771011,NaN,東京都
2,Italian Restrant & Bar Spoon,Italian Restaurant,日本橋茅場町2-4-3,35.678733,139.778021,NaN,東京都
3,Jazz & Cafe Bar Small Hours,Bar,神田神保町1-62-4,35.696879,139.757973,101-0051,東京都
4,CURRY&SOUTH ASIAN BAR SHAGORIKA,Asian Restaurant,NaN,35.695417,139.763466,NaN,東京都
5,アジアンダイニング&Bar Sathi（サティ）秋葉原店,Indian Restaurant,東京都千代田区神田佐久間町2-11,35.697975,139.776128,NaN,東京都
6,Cafe&bar Sampo,Café,神田須田町2-9-13,35.695682,139.774196,NaN,東京都
7,食卓 RESTAURANT&BAR SHOKUTAKU,Sake Bar,日本橋箱崎町16-8,35.680185,139.785211,103-0015,東京都
8,HAMACHO DINING&BAR SESSiON,Restaurant,日本橋浜町3-20-6,35.684705,139.789222,103-0007,東京都
9,Restaurant & Bar Sanmi,Molecular Gastronomy Restaurant,赤坂3丁目,35.675627,139.737121,107-0052,東京都


In [46]:
# delete rows with none values
df_bars = clean_bars_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_bars

,name,categories,address,lat,lng,postalCode,state
0,Darts&Bar share,Bar,上野2-1-9,35.708335,139.772689,110-0005,東京都
3,Jazz & Cafe Bar Small Hours,Bar,神田神保町1-62-4,35.696879,139.757973,101-0051,東京都
7,食卓 RESTAURANT&BAR SHOKUTAKU,Sake Bar,日本橋箱崎町16-8,35.680185,139.785211,103-0015,東京都
8,HAMACHO DINING&BAR SESSiON,Restaurant,日本橋浜町3-20-6,35.684705,139.789222,103-0007,東京都
9,Restaurant & Bar Sanmi,Molecular Gastronomy Restaurant,赤坂3丁目,35.675627,139.737121,107-0052,東京都
10,LIVE & BAR SECOND STEP,Bar,神田美土代町11-2,35.693620,139.766431,101-0053,東京都
11,Gallery&Bar .S,Bar,向丘2-17-21,35.720359,139.756130,113-0023,東京都
13,肉&魚Bar SANKAKU,Restaurant,余丁町4-13,35.695576,139.715820,162-0055,東京都
15,Cafe & Bar SHIRO,Café,白河1-3-4,35.681092,139.799473,135-0021,東京都
16,Asian Restaurant & Bar SAKURA,Indian Restaurant,新宿7-12-2,35.697891,139.712013,160-0022,東京都


## Looking for Restaurants

In [48]:
# search for Restaurants
search_query = 'Restaurant'
radius = 5000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=YDDCQN3JYZL2WA41EACJN2XKXPMXJLA4ISAAUS4JTIINWTMA&client_secret=DQZVHL5SPJOZ1SVDMGQEMA5QM5X5XE1SUIENWGHIWXT1N0WB&ll=35.6828387,139.7594549&v=20180605&query=Restaurant&radius=5000&limit=30'

In [0]:
# Send the GET Request and examine the results
Rresults = requests.get(url).json()
#Rresults

In [50]:
# assign relevant part of JSON to venues
venues = Rresults['response']['venues']

# tranform venues into a dataframe
Restaurant_dataframe = json_normalize(venues)
Restaurant_dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,venuePage.id
0,4ba75cb6f964a520348e39e3,Coca Restaurant x mango tree cafe,"[{'id': '4bf58dd8d48988d149941735', 'name': 'T...",v-1577152627,False,有楽町1-11-1,ビックカメラ有楽町店 6F,35.675548,139.763153,"[{'label': 'display', 'lat': 35.67554834466681...",877,100-0006,JP,東京,東京都,日本,"[有楽町1-11-1 (ビックカメラ有楽町店 6F), 千代田区, 東京都, 100-000...",NaN,NaN
1,548905d2498e81f689e1deec,Tokyo Athlete Restaurant (東京アスリート食堂),"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",v-1577152627,False,丸の内3-1-1,国際ビルヂング B1F,35.676799,139.761681,"[{'label': 'display', 'lat': 35.67679949836416...",701,100-0005,JP,東京,東京都,日本,"[丸の内3-1-1 (国際ビルヂング B1F), 千代田区, 東京都, 100-0005, 日本]",丸の内,NaN
2,55482b82498ead95e681b34b,MOTIF RESTAURANT & BAR,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1577152627,False,丸の内1-11-1,フォーシーズンズホテル丸の内 東京 7F,35.678271,139.766827,"[{'label': 'display', 'lat': 35.678271, 'lng':...",838,100-6277,JP,千代田区,東京都,日本,"[丸の内1-11-1 (フォーシーズンズホテル丸の内 東京 7F), 千代田区, 東京都, ...",NaN,NaN
3,5386a5c8498ee4b654b4ce58,Tokyo Athlete Restaurant (東京アスリート食堂),"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",v-1577152627,False,神田錦町3-21,10over9 1F,35.691580,139.761351,"[{'label': 'display', 'lat': 35.69157983304175...",988,101-0054,JP,千代田区,東京都,日本,"[神田錦町3-21 (10over9 1F), 千代田区, 東京都, 101-0054, 日本]",NaN,NaN
4,550cd716498ebe3e12462db0,The Restaurant by AMAN (ザ･レストラン by アマン),"[{'id': '4bf58dd8d48988d1c0941735', 'name': 'M...",v-1577152627,False,大手町1-5-6,大手町タワー,35.685244,139.765400,"[{'label': 'display', 'lat': 35.68524351388159...",600,NaN,JP,東京,東京都,日本,"[大手町1-5-6 (大手町タワー), 千代田区, 東京都, 日本]",NaN,NaN


### Cleaning Dataframe

In [51]:
Restaurant_clean_columns = ['name', 'categories'] + [col for col in Restaurant_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Restaurant_dataframe = Restaurant_dataframe.loc[:,Restaurant_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list3 = row['categories']
    except:
        categories_list3 = row['venue.categories']
        
    if len(categories_list3) == 0:
        return None
    else:
        return categories_list3[0]['name']

# filter the category for each row
clean_Restaurant_dataframe['categories'] = clean_Restaurant_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Restaurant_dataframe.columns = [column.split('.')[-1] for column in clean_Restaurant_dataframe.columns]

clean_Restaurant_dataframe.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Coca Restaurant x mango tree cafe,Thai Restaurant,有楽町1-11-1,ビックカメラ有楽町店 6F,35.675548,139.763153,"[{'label': 'display', 'lat': 35.67554834466681...",877,100-0006,JP,東京,東京都,日本,"[有楽町1-11-1 (ビックカメラ有楽町店 6F), 千代田区, 東京都, 100-000...",NaN,4ba75cb6f964a520348e39e3
1,Tokyo Athlete Restaurant (東京アスリート食堂),Japanese Restaurant,丸の内3-1-1,国際ビルヂング B1F,35.676799,139.761681,"[{'label': 'display', 'lat': 35.67679949836416...",701,100-0005,JP,東京,東京都,日本,"[丸の内3-1-1 (国際ビルヂング B1F), 千代田区, 東京都, 100-0005, 日本]",丸の内,548905d2498e81f689e1deec
2,MOTIF RESTAURANT & BAR,Restaurant,丸の内1-11-1,フォーシーズンズホテル丸の内 東京 7F,35.678271,139.766827,"[{'label': 'display', 'lat': 35.678271, 'lng':...",838,100-6277,JP,千代田区,東京都,日本,"[丸の内1-11-1 (フォーシーズンズホテル丸の内 東京 7F), 千代田区, 東京都, ...",NaN,55482b82498ead95e681b34b
3,Tokyo Athlete Restaurant (東京アスリート食堂),Japanese Restaurant,神田錦町3-21,10over9 1F,35.691580,139.761351,"[{'label': 'display', 'lat': 35.69157983304175...",988,101-0054,JP,千代田区,東京都,日本,"[神田錦町3-21 (10over9 1F), 千代田区, 東京都, 101-0054, 日本]",NaN,5386a5c8498ee4b654b4ce58
4,The Restaurant by AMAN (ザ･レストラン by アマン),Mediterranean Restaurant,大手町1-5-6,大手町タワー,35.685244,139.765400,"[{'label': 'display', 'lat': 35.68524351388159...",600,NaN,JP,東京,東京都,日本,"[大手町1-5-6 (大手町タワー), 千代田区, 東京都, 日本]",NaN,550cd716498ebe3e12462db0


In [52]:
# delete unnecessary columns
clean_Restaurant_dataframe2= clean_Restaurant_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)
clean_Restaurant_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Coca Restaurant x mango tree cafe,Thai Restaurant,有楽町1-11-1,35.675548,139.763153,100-0006,東京都
1,Tokyo Athlete Restaurant (東京アスリート食堂),Japanese Restaurant,丸の内3-1-1,35.676799,139.761681,100-0005,東京都
2,MOTIF RESTAURANT & BAR,Restaurant,丸の内1-11-1,35.678271,139.766827,100-6277,東京都
3,Tokyo Athlete Restaurant (東京アスリート食堂),Japanese Restaurant,神田錦町3-21,35.691580,139.761351,101-0054,東京都
4,The Restaurant by AMAN (ザ･レストラン by アマン),Mediterranean Restaurant,大手町1-5-6,35.685244,139.765400,NaN,東京都
5,Restaurant Rosette (レストラン ロゼット),French Restaurant,丸の内2-1-1,35.678964,139.761636,100-0005,東京都
6,Restaurant ベラージュ,French Restaurant,大手町2-2-2,35.685849,139.767154,100-0004,東京都
7,RESTAURANT Air (レストラン エール),French Restaurant,銀座5-7-10,35.670477,139.764262,NaN,東京都
8,Restaurant Latin (レストラン ラタン),French Restaurant,神田錦町3-28,35.692905,139.759211,101-8459,東京都
9,GREEN TEA RESTAURANT 1899 OCHANOMIZU,Tea Room,神田駿河台3-4,35.696699,139.765474,101-0062,東京都


In [53]:
# delete rows with none values
df_Restaurant = clean_Restaurant_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Restaurant

,name,categories,address,lat,lng,postalCode,state
0,Coca Restaurant x mango tree cafe,Thai Restaurant,有楽町1-11-1,35.675548,139.763153,100-0006,東京都
1,Tokyo Athlete Restaurant (東京アスリート食堂),Japanese Restaurant,丸の内3-1-1,35.676799,139.761681,100-0005,東京都
2,MOTIF RESTAURANT & BAR,Restaurant,丸の内1-11-1,35.678271,139.766827,100-6277,東京都
3,Tokyo Athlete Restaurant (東京アスリート食堂),Japanese Restaurant,神田錦町3-21,35.691580,139.761351,101-0054,東京都
5,Restaurant Rosette (レストラン ロゼット),French Restaurant,丸の内2-1-1,35.678964,139.761636,100-0005,東京都
6,Restaurant ベラージュ,French Restaurant,大手町2-2-2,35.685849,139.767154,100-0004,東京都
8,Restaurant Latin (レストラン ラタン),French Restaurant,神田錦町3-28,35.692905,139.759211,101-8459,東京都
9,GREEN TEA RESTAURANT 1899 OCHANOMIZU,Tea Room,神田駿河台3-4,35.696699,139.765474,101-0062,東京都
10,Restaurant Luke with Sky Lounge,French Restaurant,明石町8-1,35.667025,139.778621,104-0044,東京都
11,Restaurant Platinum Fish (Restaurant 白金魚 Plati...,Restaurant,神田須田町1-25-4,35.697369,139.769888,101-0041,東京都


##Looking for Tourist Information Center

In [56]:
# search for Tourist Places
search_query = 'Tourist Center'
radius = 5000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=YDDCQN3JYZL2WA41EACJN2XKXPMXJLA4ISAAUS4JTIINWTMA&client_secret=DQZVHL5SPJOZ1SVDMGQEMA5QM5X5XE1SUIENWGHIWXT1N0WB&ll=35.6828387,139.7594549&v=20180605&query=Tourist Center&radius=5000&limit=30'

In [0]:
# Send the GET Request and examine the results
tou_results = requests.get(url).json()

In [58]:
# assign relevant part of JSON to venues
venues = tou_results['response']['venues']

# tranform venues into a dataframe
tou_dataframe = json_normalize(venues)
tou_dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.postalCode,location.neighborhood
0,5115a84fe4b05931d8da857e,Tourist Information Center - 日本政府観光局 (JNTO),"[{'id': '4f4530164b9074f6e4fb00ff', 'name': 'T...",v-1577152668,False,丸の内3-3-1,新東京ビル 1F,35.677607,139.762545,"[{'label': 'display', 'lat': 35.67760739874021...",645,JP,東京,東京都,日本,"[丸の内3-3-1 (新東京ビル 1F), 千代田区, 東京都, 日本]",NaN,NaN
1,4c5a8200f346c928682d4fca,TIC東京 (Tourist Information Center Tokyo),"[{'id': '4f4530164b9074f6e4fb00ff', 'name': 'T...",v-1577152668,False,丸の内1-8-1,丸の内トラストタワーN館 1F,35.683260,139.769661,"[{'label': 'display', 'lat': 35.68326, 'lng': ...",924,JP,東京,東京都,日本,"[丸の内1-8-1 (丸の内トラストタワーN館 1F), 千代田区, 東京都, 日本]",NaN,NaN
2,4e40a74ee4cd4324642f4914,Asakusa Culture Tourist Information Center (浅草...,"[{'id': '4f4530164b9074f6e4fb00ff', 'name': 'T...",v-1577152668,False,雷門2-18-9,NaN,35.710820,139.796405,"[{'label': 'display', 'lat': 35.71081989121610...",4567,JP,東京,東京都,日本,"[雷門2-18-9, 台東区, 東京都, 111-0034, 日本]",111-0034,NaN
3,4b555139f964a5207ae127e3,Shiodome City Center (汐留シティセンター),"[{'id': '56aa371be4b08b9a8d573517', 'name': 'B...",v-1577152668,False,東新橋1-5-2,NaN,35.665350,139.760813,"[{'label': 'display', 'lat': 35.66534990209759...",1950,JP,港区,東京都,日本,"[東新橋1-5-2, 港区, 東京都, 105-7108, 日本]",105-7108,NaN
4,4cc1135721a4f04d46106255,Tokyo Tourist Information Center (東京観光情報センター 都...,"[{'id': '4f4530164b9074f6e4fb00ff', 'name': 'T...",v-1577152668,False,西新宿2-8-1,東京都庁 第一本庁舎 1F,35.689821,139.691714,"[{'label': 'display', 'lat': 35.68982052002047...",6173,JP,東京,東京都,日本,"[西新宿2-8-1 (東京都庁 第一本庁舎 1F), 新宿区, 東京都, 163-8001,...",163-8001,西新宿


#Clean Dataframe

In [59]:
# keep only columns that include venue name, and anything that is associated with location
tou_clean_columns = ['name', 'categories'] + [col for col in tou_dataframe.columns if col.startswith('location.')]+ ['id']
clean_tou_dataframe = tou_dataframe.loc[:, tou_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list1 = row['categories']
    except:
        categories_list1 = row['venue.categories']
        
    if len(categories_list1) == 0:
        return None
    else:
        return categories_list1[0]['name']

# filter the category for each row
clean_tou_dataframe['categories'] = clean_tou_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_tou_dataframe.columns = [column.split('.')[-1] for column in clean_tou_dataframe.columns]

clean_tou_dataframe.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,postalCode,neighborhood,id
0,Tourist Information Center - 日本政府観光局 (JNTO),Tourist Information Center,丸の内3-3-1,新東京ビル 1F,35.677607,139.762545,"[{'label': 'display', 'lat': 35.67760739874021...",645,JP,東京,東京都,日本,"[丸の内3-3-1 (新東京ビル 1F), 千代田区, 東京都, 日本]",NaN,NaN,5115a84fe4b05931d8da857e
1,TIC東京 (Tourist Information Center Tokyo),Tourist Information Center,丸の内1-8-1,丸の内トラストタワーN館 1F,35.683260,139.769661,"[{'label': 'display', 'lat': 35.68326, 'lng': ...",924,JP,東京,東京都,日本,"[丸の内1-8-1 (丸の内トラストタワーN館 1F), 千代田区, 東京都, 日本]",NaN,NaN,4c5a8200f346c928682d4fca
2,Asakusa Culture Tourist Information Center (浅草...,Tourist Information Center,雷門2-18-9,NaN,35.710820,139.796405,"[{'label': 'display', 'lat': 35.71081989121610...",4567,JP,東京,東京都,日本,"[雷門2-18-9, 台東区, 東京都, 111-0034, 日本]",111-0034,NaN,4e40a74ee4cd4324642f4914
3,Shiodome City Center (汐留シティセンター),Business Center,東新橋1-5-2,NaN,35.665350,139.760813,"[{'label': 'display', 'lat': 35.66534990209759...",1950,JP,港区,東京都,日本,"[東新橋1-5-2, 港区, 東京都, 105-7108, 日本]",105-7108,NaN,4b555139f964a5207ae127e3
4,Tokyo Tourist Information Center (東京観光情報センター 都...,Tourist Information Center,西新宿2-8-1,東京都庁 第一本庁舎 1F,35.689821,139.691714,"[{'label': 'display', 'lat': 35.68982052002047...",6173,JP,東京,東京都,日本,"[西新宿2-8-1 (東京都庁 第一本庁舎 1F), 新宿区, 東京都, 163-8001,...",163-8001,西新宿,4cc1135721a4f04d46106255


In [60]:
# delete unnecessary columns
clean_tou_dataframe2= clean_tou_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)
clean_tou_dataframe2

,name,categories,address,lat,lng,state,postalCode
0,Tourist Information Center - 日本政府観光局 (JNTO),Tourist Information Center,丸の内3-3-1,35.677607,139.762545,東京都,NaN
1,TIC東京 (Tourist Information Center Tokyo),Tourist Information Center,丸の内1-8-1,35.683260,139.769661,東京都,NaN
2,Asakusa Culture Tourist Information Center (浅草...,Tourist Information Center,雷門2-18-9,35.710820,139.796405,東京都,111-0034
3,Shiodome City Center (汐留シティセンター),Business Center,東新橋1-5-2,35.665350,139.760813,東京都,105-7108
4,Tokyo Tourist Information Center (東京観光情報センター 都...,Tourist Information Center,西新宿2-8-1,35.689821,139.691714,東京都,163-8001
5,Tourist Information Center Ginza,Tourist Information Center,銀座4-1,35.673167,139.762460,東京都,104-0061
6,World Trade Center Building (世界貿易センタービルディング),Business Center,浜松町2-4-1,35.656115,139.756570,東京都,105ｰ6140
7,Tourist Information Center Ginza Core,Tourist Information Center,銀座5-8-20,35.670855,139.764858,東京都,104-0061
8,National Cancer Center (国立がん研究センター),Medical Center,築地5-1-1,35.665315,139.767720,東京都,104-0045
9,佐川急便 東京手ぶら観光手荷物預かり処 TOKYO SERVICE CENTER,Shipping Store,丸の内1‐9‐1,35.683373,139.769084,東京都,100-0005


In [61]:
# delete rows with none values
df_tourist = clean_tou_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_tourist

,name,categories,address,lat,lng,state,postalCode
2,Asakusa Culture Tourist Information Center (浅草...,Tourist Information Center,雷門2-18-9,35.710820,139.796405,東京都,111-0034
3,Shiodome City Center (汐留シティセンター),Business Center,東新橋1-5-2,35.665350,139.760813,東京都,105-7108
4,Tokyo Tourist Information Center (東京観光情報センター 都...,Tourist Information Center,西新宿2-8-1,35.689821,139.691714,東京都,163-8001
5,Tourist Information Center Ginza,Tourist Information Center,銀座4-1,35.673167,139.762460,東京都,104-0061
6,World Trade Center Building (世界貿易センタービルディング),Business Center,浜松町2-4-1,35.656115,139.756570,東京都,105ｰ6140
7,Tourist Information Center Ginza Core,Tourist Information Center,銀座5-8-20,35.670855,139.764858,東京都,104-0061
8,National Cancer Center (国立がん研究センター),Medical Center,築地5-1-1,35.665315,139.767720,東京都,104-0045
9,佐川急便 東京手ぶら観光手荷物預かり処 TOKYO SERVICE CENTER,Shipping Store,丸の内1‐9‐1,35.683373,139.769084,東京都,100-0005
10,The National Art Center (国立新美術館),Art Museum,六本木7-22-2,35.665133,139.726417,東京都,106-8558
11,Pokémon Center Shibuya (ポケモンセンターシブヤ),Hobby Shop,宇田川町15-1,35.662131,139.698986,東京都,150-8377


##Search for Spa

In [62]:
# search for spa
search_query = 'Spa'
radius = 2000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=YDDCQN3JYZL2WA41EACJN2XKXPMXJLA4ISAAUS4JTIINWTMA&client_secret=DQZVHL5SPJOZ1SVDMGQEMA5QM5X5XE1SUIENWGHIWXT1N0WB&ll=35.6828387,139.7594549&v=20180605&query=Spa&radius=2000&limit=30'

In [0]:
# Send the GET Request and examine the results
results = requests.get(url).json()
#results

In [64]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood
0,5c73a3c8b9a5a8002c869999,RAKU SPA 1010,"[{'id': '52e81612bcbc57f1066b7a27', 'name': 'B...",v-1577152727,False,神田淡路町2-9-9,35.698097,139.767908,"[{'label': 'display', 'lat': 35.698097, 'lng':...",1862,101-0063,JP,千代田区,東京都,日本,"[神田淡路町2-9-9, 千代田区, 東京都, 101-0063, 日本]",NaN,NaN
1,550cd87f498ed2931e8748ae,AMAN SPA (アマン スパ),"[{'id': '4bf58dd8d48988d1ed941735', 'name': 'S...",v-1577152727,False,大手町1-5-6,35.685448,139.765406,"[{'label': 'display', 'lat': 35.68544840454873...",611,NaN,JP,東京,東京都,日本,"[大手町1-5-6 (アマン東京), 千代田区, 東京都, 日本]",アマン東京,NaN
2,4f6d8017e4b07a4bc474ab22,THE PENINSULA SPA (ザ・ペニンシュラ スパ),"[{'id': '4bf58dd8d48988d1ed941735', 'name': 'S...",v-1577152727,False,有楽町1-8-1,35.674840,139.760259,"[{'label': 'display', 'lat': 35.67483965691738...",893,100-0006,JP,東京,東京都,日本,"[有楽町1-8-1 (ザ・ペニンシュラ東京 6F), 千代田区, 東京都, 100-0006...",ザ・ペニンシュラ東京 6F,NaN
3,4b0587a6f964a520459e22e3,Spa at Four Seasons Hotel Tokyo at Marunouchi,"[{'id': '4bf58dd8d48988d1ed941735', 'name': 'S...",v-1577152727,False,丸の内1-11-1,35.677998,139.767277,"[{'label': 'display', 'lat': 35.67799814146347...",889,100-6277,JP,千代田区,東京都,日本,"[丸の内1-11-1 (パシフィックセンチュリープレイス), 千代田区, 東京都, 100-...",パシフィックセンチュリープレイス,NaN
4,4dbfdd0b4df0f8fd6b8ef5a0,Atago Green Hills Spa (愛宕グリーンヒルズスパ),"[{'id': '4bf58dd8d48988d1ed941735', 'name': 'S...",v-1577152727,False,愛宕2-3-1,35.663464,139.748972,"[{'label': 'display', 'lat': 35.66346436983380...",2355,105-0002,JP,東京,東京都,日本,"[愛宕2-3-1 (愛宕グリーンヒルズフォレストタワー 42F), 港区, 東京都, 105...",愛宕グリーンヒルズフォレストタワー 42F,NaN


##Clean Dataframe

In [65]:
# keep only columns that include venue name, and anything that is associated with location
clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_dataframe = dataframe.loc[:,clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
clean_dataframe['categories'] = clean_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_dataframe.columns = [column.split('.')[-1] for column in clean_dataframe.columns]

clean_dataframe.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,RAKU SPA 1010,Bath House,神田淡路町2-9-9,35.698097,139.767908,"[{'label': 'display', 'lat': 35.698097, 'lng':...",1862,101-0063,JP,千代田区,東京都,日本,"[神田淡路町2-9-9, 千代田区, 東京都, 101-0063, 日本]",NaN,NaN,5c73a3c8b9a5a8002c869999
1,AMAN SPA (アマン スパ),Spa,大手町1-5-6,35.685448,139.765406,"[{'label': 'display', 'lat': 35.68544840454873...",611,NaN,JP,東京,東京都,日本,"[大手町1-5-6 (アマン東京), 千代田区, 東京都, 日本]",アマン東京,NaN,550cd87f498ed2931e8748ae
2,THE PENINSULA SPA (ザ・ペニンシュラ スパ),Spa,有楽町1-8-1,35.674840,139.760259,"[{'label': 'display', 'lat': 35.67483965691738...",893,100-0006,JP,東京,東京都,日本,"[有楽町1-8-1 (ザ・ペニンシュラ東京 6F), 千代田区, 東京都, 100-0006...",ザ・ペニンシュラ東京 6F,NaN,4f6d8017e4b07a4bc474ab22
3,Spa at Four Seasons Hotel Tokyo at Marunouchi,Spa,丸の内1-11-1,35.677998,139.767277,"[{'label': 'display', 'lat': 35.67799814146347...",889,100-6277,JP,千代田区,東京都,日本,"[丸の内1-11-1 (パシフィックセンチュリープレイス), 千代田区, 東京都, 100-...",パシフィックセンチュリープレイス,NaN,4b0587a6f964a520459e22e3
4,Atago Green Hills Spa (愛宕グリーンヒルズスパ),Spa,愛宕2-3-1,35.663464,139.748972,"[{'label': 'display', 'lat': 35.66346436983380...",2355,105-0002,JP,東京,東京都,日本,"[愛宕2-3-1 (愛宕グリーンヒルズフォレストタワー 42F), 港区, 東京都, 105...",愛宕グリーンヒルズフォレストタワー 42F,NaN,4dbfdd0b4df0f8fd6b8ef5a0


In [66]:
clean_dataframe= clean_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs','neighborhood', 'id'], axis=1)
clean_dataframe

,name,categories,address,lat,lng,postalCode,state
0,RAKU SPA 1010,Bath House,神田淡路町2-9-9,35.698097,139.767908,101-0063,東京都
1,AMAN SPA (アマン スパ),Spa,大手町1-5-6,35.685448,139.765406,NaN,東京都
2,THE PENINSULA SPA (ザ・ペニンシュラ スパ),Spa,有楽町1-8-1,35.674840,139.760259,100-0006,東京都
3,Spa at Four Seasons Hotel Tokyo at Marunouchi,Spa,丸の内1-11-1,35.677998,139.767277,100-6277,東京都
4,Atago Green Hills Spa (愛宕グリーンヒルズスパ),Spa,愛宕2-3-1,35.663464,139.748972,105-0002,東京都
5,SIECLE hair&spa (シエクル ヘア アンド スパ),Salon / Barbershop,銀座8-9-15,35.668094,139.761865,104-0061,東京都
6,SPA大手町,Gym / Fitness Center,大手町1-9-2,35.688315,139.764494,100-0004,東京都
7,"The Spa at Mandarin Oriental, Tokyo",Spa,2-1-1 Nihonbashi Muromachi,35.687010,139.773056,103-8328,東京都
8,evian Spa,Spa,丸の内1-1-1,35.684873,139.761077,100-0005,東京都
9,Natural Design SPA (ナチュラルデザインスパ),Nail Salon,銀座2-11-14,35.672089,139.768669,104-0061,東京都


In [67]:
# delete rows with none values
df_spa = clean_dataframe.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_spa

,name,categories,address,lat,lng,postalCode,state
0,RAKU SPA 1010,Bath House,神田淡路町2-9-9,35.698097,139.767908,101-0063,東京都
2,THE PENINSULA SPA (ザ・ペニンシュラ スパ),Spa,有楽町1-8-1,35.674840,139.760259,100-0006,東京都
3,Spa at Four Seasons Hotel Tokyo at Marunouchi,Spa,丸の内1-11-1,35.677998,139.767277,100-6277,東京都
4,Atago Green Hills Spa (愛宕グリーンヒルズスパ),Spa,愛宕2-3-1,35.663464,139.748972,105-0002,東京都
5,SIECLE hair&spa (シエクル ヘア アンド スパ),Salon / Barbershop,銀座8-9-15,35.668094,139.761865,104-0061,東京都
6,SPA大手町,Gym / Fitness Center,大手町1-9-2,35.688315,139.764494,100-0004,東京都
7,"The Spa at Mandarin Oriental, Tokyo",Spa,2-1-1 Nihonbashi Muromachi,35.687010,139.773056,103-8328,東京都
8,evian Spa,Spa,丸の内1-1-1,35.684873,139.761077,100-0005,東京都
9,Natural Design SPA (ナチュラルデザインスパ),Nail Salon,銀座2-11-14,35.672089,139.768669,104-0061,東京都
12,The Peninsula Tokyo (ザ・ペニンシュラ東京),Hotel,有楽町1-8-1,35.674724,139.760553,100-0006,東京都


##For Parks

In [68]:
# search for Parks
search_query = 'Park'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=YDDCQN3JYZL2WA41EACJN2XKXPMXJLA4ISAAUS4JTIINWTMA&client_secret=DQZVHL5SPJOZ1SVDMGQEMA5QM5X5XE1SUIENWGHIWXT1N0WB&ll=35.6828387,139.7594549&v=20180605&query=Park&radius=10000&limit=30'

In [0]:
# Send the GET Request and examine the results
presults = requests.get(url).json()
#presults

In [70]:
# assign relevant part of JSON to venues
venues = presults['response']['venues']

# tranform venues into a dataframe
park_dataframe = json_normalize(venues)
park_dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,location.crossStreet
0,4b564bf1f964a520da0928e3,Hibiya Park (日比谷公園),"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1577152763,False,日比谷公園1,35.673692,139.756173,"[{'label': 'display', 'lat': 35.67369194669595...",1060,100-0012,JP,東京,東京都,日本,"[日比谷公園1, 千代田区, 東京都, 100-0012, 日本]",NaN,NaN
1,4b0587a1f964a5209b9d22e3,Ueno Park (上野恩賜公園),"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1577152763,False,上野公園/池之端3,35.714675,139.773487,"[{'label': 'display', 'lat': 35.71467517039568...",3764,NaN,JP,東京,東京都,日本,"[上野公園/池之端3, 台東区, 東京都, 日本]",NaN,NaN
2,4b5a3a54f964a520cdb528e3,Yoyogi Park (代々木公園),"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1577152763,False,代々木神園町/神南2,35.670437,139.695671,"[{'label': 'display', 'lat': 35.67043660939139...",5930,151-0052,JP,東京,東京都,日本,"[代々木神園町/神南2, 渋谷区, 東京都, 151-0052, 日本]",NaN,NaN
3,4bb6a78746d4a59398a0c6c0,Shiba Park (芝公園),"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1577152763,False,芝公園,35.656366,139.748041,"[{'label': 'display', 'lat': 35.65636636950145...",3122,105-0011,JP,東京,東京都,日本,"[芝公園, 港区, 東京都, 105-0011, 日本]",NaN,NaN
4,4b870e83f964a5200cae31e3,The Prince Park Tower Tokyo (ザ・プリンス パークタワー東京),"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1577152763,False,芝公園4-8-1,35.655467,139.747073,"[{'label': 'display', 'lat': 35.65546666809256...",3246,105-8563,JP,東京,東京都,日本,"[芝公園4-8-1, 港区, 東京都, 105-8563, 日本]",NaN,NaN


##Cleaning Dataframe

In [71]:
# keep only columns that include venue name, and anything that is associated with location
park_clean_columns = ['name', 'categories'] + [col for col in park_dataframe.columns if col.startswith('location.')]+ ['id']
clean_park_dataframe = park_dataframe.loc[:,park_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list1 = row['categories']
    except:
        categories_list1 = row['venue.categories']
        
    if len(categories_list1) == 0:
        return None
    else:
        return categories_list1[0]['name']

# filter the category for each row
clean_park_dataframe['categories'] = clean_park_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_park_dataframe.columns = [column.split('.')[-1] for column in clean_park_dataframe.columns]

clean_park_dataframe.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,crossStreet,id
0,Hibiya Park (日比谷公園),Park,日比谷公園1,35.673692,139.756173,"[{'label': 'display', 'lat': 35.67369194669595...",1060,100-0012,JP,東京,東京都,日本,"[日比谷公園1, 千代田区, 東京都, 100-0012, 日本]",NaN,NaN,4b564bf1f964a520da0928e3
1,Ueno Park (上野恩賜公園),Park,上野公園/池之端3,35.714675,139.773487,"[{'label': 'display', 'lat': 35.71467517039568...",3764,NaN,JP,東京,東京都,日本,"[上野公園/池之端3, 台東区, 東京都, 日本]",NaN,NaN,4b0587a1f964a5209b9d22e3
2,Yoyogi Park (代々木公園),Park,代々木神園町/神南2,35.670437,139.695671,"[{'label': 'display', 'lat': 35.67043660939139...",5930,151-0052,JP,東京,東京都,日本,"[代々木神園町/神南2, 渋谷区, 東京都, 151-0052, 日本]",NaN,NaN,4b5a3a54f964a520cdb528e3
3,Shiba Park (芝公園),Park,芝公園,35.656366,139.748041,"[{'label': 'display', 'lat': 35.65636636950145...",3122,105-0011,JP,東京,東京都,日本,"[芝公園, 港区, 東京都, 105-0011, 日本]",NaN,NaN,4bb6a78746d4a59398a0c6c0
4,The Prince Park Tower Tokyo (ザ・プリンス パークタワー東京),Hotel,芝公園4-8-1,35.655467,139.747073,"[{'label': 'display', 'lat': 35.65546666809256...",3246,105-8563,JP,東京,東京都,日本,"[芝公園4-8-1, 港区, 東京都, 105-8563, 日本]",NaN,NaN,4b870e83f964a5200cae31e3


In [72]:
# delete unnecessary columns
clean_park_dataframe2= clean_park_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)
clean_park_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Hibiya Park (日比谷公園),Park,日比谷公園1,35.673692,139.756173,100-0012,東京都
1,Ueno Park (上野恩賜公園),Park,上野公園/池之端3,35.714675,139.773487,NaN,東京都
2,Yoyogi Park (代々木公園),Park,代々木神園町/神南2,35.670437,139.695671,151-0052,東京都
3,Shiba Park (芝公園),Park,芝公園,35.656366,139.748041,105-0011,東京都
4,The Prince Park Tower Tokyo (ザ・プリンス パークタワー東京),Hotel,芝公園4-8-1,35.655467,139.747073,105-8563,東京都
5,Hakuhinkan Toy Park (博品館 TOY PARK 銀座本店),Toy / Game Store,銀座8-8-11,35.667832,139.761034,104-8132,東京都
6,TOKYO FM Ginza Sony Park Studio,Radio Station,銀座5-3-1,35.672573,139.763209,104-0061,東京都
7,Shinjuku Chuo Park (新宿中央公園),Park,西新宿2-11,35.689397,139.689600,160-0023,東京都
8,Park Hyatt Tokyo (パークハイアット東京),Hotel,西新宿3-7-1-2,35.685575,139.690639,163-1055,東京都
9,Kasai-Rinkai Park Station (葛西臨海公園駅),Train Station,臨海町6-3-3,35.644083,139.862254,134-0086,東京都


In [73]:
# delete rows with none values
clean_park_dataframe3 = clean_park_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_park_dataframe3

,name,categories,address,lat,lng,postalCode,state
0,Hibiya Park (日比谷公園),Park,日比谷公園1,35.673692,139.756173,100-0012,東京都
2,Yoyogi Park (代々木公園),Park,代々木神園町/神南2,35.670437,139.695671,151-0052,東京都
3,Shiba Park (芝公園),Park,芝公園,35.656366,139.748041,105-0011,東京都
4,The Prince Park Tower Tokyo (ザ・プリンス パークタワー東京),Hotel,芝公園4-8-1,35.655467,139.747073,105-8563,東京都
5,Hakuhinkan Toy Park (博品館 TOY PARK 銀座本店),Toy / Game Store,銀座8-8-11,35.667832,139.761034,104-8132,東京都
6,TOKYO FM Ginza Sony Park Studio,Radio Station,銀座5-3-1,35.672573,139.763209,104-0061,東京都
7,Shinjuku Chuo Park (新宿中央公園),Park,西新宿2-11,35.689397,139.689600,160-0023,東京都
8,Park Hyatt Tokyo (パークハイアット東京),Hotel,西新宿3-7-1-2,35.685575,139.690639,163-1055,東京都
9,Kasai-Rinkai Park Station (葛西臨海公園駅),Train Station,臨海町6-3-3,35.644083,139.862254,134-0086,東京都
10,Hinokicho Park (檜町公園),Park,赤坂9-7-9,35.666798,139.732586,107-0052,東京都


In [74]:
# delete rows which its category is not Park
df_park = clean_park_dataframe3[clean_park_dataframe3.categories == 'Park']
df_park

,name,categories,address,lat,lng,postalCode,state
0,Hibiya Park (日比谷公園),Park,日比谷公園1,35.673692,139.756173,100-0012,東京都
2,Yoyogi Park (代々木公園),Park,代々木神園町/神南2,35.670437,139.695671,151-0052,東京都
3,Shiba Park (芝公園),Park,芝公園,35.656366,139.748041,105-0011,東京都
7,Shinjuku Chuo Park (新宿中央公園),Park,西新宿2-11,35.689397,139.689600,160-0023,東京都
10,Hinokicho Park (檜町公園),Park,赤坂9-7-9,35.666798,139.732586,107-0052,東京都
11,Odaiba Marine Park (お台場海浜公園),Park,台場1-4-1,35.631775,139.777733,135-0091,東京都
12,Kasai Rinkai Park (葛西臨海公園),Park,臨海町6,35.642998,139.860334,134-8587,東京都
13,Minami-Ikebukuro Park (南池袋公園),Park,南池袋2-21-1,35.727593,139.714336,171-0022,東京都
17,御徒町公園 (Okachimachi Park) (御徒町公園),Park,台東4-13-3,35.706209,139.777649,110-0016,東京都
27,Aoyama Park (青山公園),Park,南青山2-21-12,35.669627,139.724570,107-0062,東京都


#Generating Map

In [76]:
# create dataframe 
df = pd.concat([df_hospital, df_hotels, df_Cafeteria, df_Shopping, df_bars, df_Restaurant, df_tourist, df_spa, df_park], ignore_index=True)
df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,address,categories,lat,lng,name,neighborhood,postalCode,state
0,虎ノ門2-2-2,Hospital,35.668780,139.746678,Toranomon Hospital (虎の門病院),NaN,105-0001,NaN
1,千代田1-2,Hospital,35.686930,139.759784,Hospital of the Imperial Household (宮内庁病院),NaN,100-0001,NaN
2,本郷3-1-3,Hospital,35.702272,139.762504,Juntendo University Hospital (順天堂大学医学部附属 順天堂医院),NaN,113-8431,NaN
3,築地5-1-1,Hospital,35.665488,139.768159,National Cancer Center Hospital (国立がん研究センター中央病院),NaN,104-0045,NaN
4,西新橋3-19-18,Hospital,35.663503,139.751265,Jikei University School of Medicine Hospital (...,NaN,105-8471,NaN
...,...,...,...,...,...,...,...,...
133,台場1-4-1,Park,35.631775,139.777733,Odaiba Marine Park (お台場海浜公園),NaN,135-0091,東京都
134,臨海町6,Park,35.642998,139.860334,Kasai Rinkai Park (葛西臨海公園),NaN,134-8587,東京都
135,南池袋2-21-1,Park,35.727593,139.714336,Minami-Ikebukuro Park (南池袋公園),NaN,171-0022,東京都
136,台東4-13-3,Park,35.706209,139.777649,御徒町公園 (Okachimachi Park) (御徒町公園),NaN,110-0016,東京都


In [88]:
# Generate map to visualize hotel neighbourhood including shopping stores and Cafeteria 
df_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(df['lat'], df['lng'], 
                                           df['name'], df['categories'],\
                                               df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='green',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(df_map)  
    
df_map

#### Map for Basic facilities like Hotels, Hospital and Restaurant

In [83]:
df2 = pd.concat([df_hospital, df_hotels, df_Restaurant], ignore_index=True)
df2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,address,categories,lat,lng,name,postalCode,state
0,虎ノ門2-2-2,Hospital,35.668780,139.746678,Toranomon Hospital (虎の門病院),105-0001,NaN
1,千代田1-2,Hospital,35.686930,139.759784,Hospital of the Imperial Household (宮内庁病院),100-0001,NaN
2,本郷3-1-3,Hospital,35.702272,139.762504,Juntendo University Hospital (順天堂大学医学部附属 順天堂医院),113-8431,NaN
3,築地5-1-1,Hospital,35.665488,139.768159,National Cancer Center Hospital (国立がん研究センター中央病院),104-0045,NaN
4,西新橋3-19-18,Hospital,35.663503,139.751265,Jikei University School of Medicine Hospital (...,105-8471,NaN
5,神田駿河台1-6,Hospital,35.697421,139.762455,Nihon University Hospital (日本大学病院),101-8309,NaN
6,神田駿河台1-8,Hospital,35.698375,139.762933,Kyoundo Hospital (杏雲堂病院),101-0062,NaN
7,麹町1-10-5,Hospital,35.685427,139.742962,Hanzomon Hospital (半蔵門病院),102-0083,NaN
8,神田和泉町1,Hospital,35.700022,139.778912,Mitsui Memorial Hospital (三井記念病院),101-8643,NaN
9,神田駿河台2-5,Hospital,35.700489,139.761500,Sanraku Hospital (三楽病院),101-0062,NaN


In [89]:
basic_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(df2['lat'], df2['lng'], 
                                           df2['name'], df2['categories'],\
                                               df2['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(basic_map)  
    
basic_map

#### Map to visualize neighbourhood including Cafeteria , Shopping Centers, Tourist Center, Spa and Park


In [90]:
df3 = pd.concat([df_Cafeteria, df_Shopping, df_bars, df_tourist, df_spa, df_park], ignore_index=True)
df3

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,address,categories,lat,lng,name,neighborhood,postalCode,state
0,豊洲3-7-5,College Cafeteria,35.660395,139.794331,Shibaura Institute of Technology Cafeteria (芝浦...,NaN,135-8548,東京都
1,西新宿2-3-2,Cafeteria,35.687701,139.695336,KDDI CAFETERIA MENDON屋,NaN,163-8003,東京都
2,六本木7-22-2,Café,35.664747,139.726748,Cafétéria Carré,NaN,106-0032,東京都
3,神田駿河台1-1,College Cafeteria,35.697456,139.761451,明治大学 スカイラウンジ暁,NaN,101-8301,東京都
4,六本木3-9-8,Café,35.663479,139.733603,Esprit kitchen Cafeteria,NaN,106-0032,東京都
...,...,...,...,...,...,...,...,...
76,台場1-4-1,Park,35.631775,139.777733,Odaiba Marine Park (お台場海浜公園),NaN,135-0091,東京都
77,臨海町6,Park,35.642998,139.860334,Kasai Rinkai Park (葛西臨海公園),NaN,134-8587,東京都
78,南池袋2-21-1,Park,35.727593,139.714336,Minami-Ikebukuro Park (南池袋公園),NaN,171-0022,東京都
79,台東4-13-3,Park,35.706209,139.777649,御徒町公園 (Okachimachi Park) (御徒町公園),NaN,110-0016,東京都


In [95]:
df3_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(df3['lat'], df3['lng'], 
                                           df3['name'], df3['categories'],\
                                               df3['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='black',
        fill=True,
        fill_color='orange',
        fill_opacity=0.7,
        parse_html=False).add_to(df3_map)  
    
df3_map